In [9]:
import sys; print('python version used: ',sys.version)

import pandas as pd
import numpy as np

import time
from prettytable import PrettyTable
from riotwatcher import LolWatcher, ApiError

%matplotlib inline
#see all columns/rows
pd.set_option('display.max_columns', None)

python version used:  3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]


In [13]:
x = PrettyTable()

print("Get 1025 games for each division's tier in League of Legends current patch.")
print("This means 1025 games for 8 divisions, and 23 tiers, or for 1025*23=23575 games")
print("Note: there's no guarentee the data will actually retain 1025/division/tier, ",
     "as some people will have changed their name since their the time of running this code.")

print("The order of rankings from lowest to highest is as below:")
x.field_names = ["division", "IV", "III", "II", "I"]
x.add_row(["IRON", "IRON IV", "IRON III", "IRON II", "IRON I"])
x.add_row(["SILVER", "SILVER IV", "SILVER III", "SILVER II", "SILVER I"])
x.add_row(["GOLD", "GOLD IV", "GOLD III", "GOLD II", "GOLD I"])
x.add_row(["PLATINUM", "PLATINUM IV", "PLATINUM III", "PLATINUM II", "PLATINUM I"])
x.add_row(["DIAMOND", "DIAMOND IV", "DIAMOND III", "DIAMOND II", "DIAMOND I"])
x.add_row(["MASTER", "MASTER I", "-", "-", "-"])
x.add_row(["GRANDMASTER", "GRANDMASTER I", "-", "-", "-"])
x.add_row(["CHALLENGER", "CHALLENGER I", "-", "-", "-"])

print(x)

print('--- LAYOUT OF THIS NOTEBOOK ---')
print('(1) Create a class "game_history_by_league" which gets us a list of 1025 players in each divison/tier')
print('(2) Create another list of all divisions/tiers')
print('(3) Find Game IDs for the last solo-queue game played by each player and add that in as a column')
print('*TAKES A FEW DAYS*')
print('(4) Run code to get match data on each of those games')
print('*TAKES A WEEK*')

Get 1025 games for each division's tier in League of Legends current patch.
This means 1025 games for 8 divisions, and 23 tiers, or for 1025*23=23575 games
Note: there's no guarentee the data will actually retain 1025/division/tier,  as some people will have changed their name since their the time of running this code.
The order of rankings from lowest to highest is as below:
+-------------+---------------+--------------+-------------+------------+
|   division  |       IV      |     III      |      II     |     I      |
+-------------+---------------+--------------+-------------+------------+
|     IRON    |    IRON IV    |   IRON III   |   IRON II   |   IRON I   |
|    SILVER   |   SILVER IV   |  SILVER III  |  SILVER II  |  SILVER I  |
|     GOLD    |    GOLD IV    |   GOLD III   |   GOLD II   |   GOLD I   |
|   PLATINUM  |  PLATINUM IV  | PLATINUM III | PLATINUM II | PLATINUM I |
|   DIAMOND   |   DIAMOND IV  | DIAMOND III  |  DIAMOND II | DIAMOND I  |
|    MASTER   |    MASTER I  

In [12]:
class game_history_by_league():

    def __init__(self, api_key, region):
        #upon calling the class we pass in a bunch of things to initialize^
        self.api_key = api_key
        self.region = region

    def get_summoners_for_each_division_tier(self):
        """
        Returns 1025 names from each ranked league in League of Legends.
        """
        watcher = LolWatcher(self.api_key, timeout=1)

        #ranked_leagues = ['CHALLENGER', 'GRANDMASTER', 'DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'IRON']
        ranked_leagues = ['DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'IRON']
        divisions = ['I', 'II', 'III', 'IV']
        solo5v5 = 'RANKED_SOLO_5x5'
        ranked_games_by_tier_by_division = {} # we will dump the JSON object returned by the API request into this dictionary

        for i in ranked_leagues:
            for j in divisions:
                #returns 1025 summoner IDs by tier by division, i.e. DIAMOND I, DIAMOND II etc.
                #first define key-value pair in dictionary (considered default dict but it's too specific)
                ranked_games_by_tier_by_division[i+' '+j] = watcher.league.entries(division=j, tier=i, queue=solo5v5, region=self.region, page=1)
                for k in range(2, 10):
                    #add onto key-value pair more games per division, it is only 205 with one page,
                    #so if we loop through this many pages we get 1025 games
                    ranked_games_by_tier_by_division[i+' '+j] += watcher.league.entries(division=j, tier=i, queue=solo5v5, region=self.region, page=k)
        return ranked_games_by_tier_by_division

    def read_account_game_ids_by_league(self):
        """
        API reference: MatchApiv4 endpoint.
        Reads match history (n=1, the last game played) for each summoner name in the lists made above for just game IDs.

        Go from ranked_games_by_tier_by_division --> game_ids_by_tier_by_division.

        """
        watcher = LolWatcher(self.api_key)

        ranked_games_by_tier_by_division = game_history_by_league(self.api_key, self.region).get_summoners_for_each_division_tier()

        game_ids_by_tier_by_division = {}

        for i in ranked_leagues:
            for j in ranked_summoners:
                watcher.matchlist_by_account(j) #this will return a matchlist_by_account

In [2]:
api_key = '' #get your own at https://developer.riotgames.com/
region = 'na1'
watcher = LolWatcher(api_key)

In [5]:
ranked_games_by_division_by_tier = game_history_by_league(api_key, region).get_summoners_for_each_division_tier()

In [6]:
ranked_leagues = ['DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'IRON']
#divisions = ['I', 'II', 'III', 'IV']
divisions = ['I', 'II', 'III', 'IV']

all_divisions = []
for i in ranked_leagues:
    for j in divisions:
        all_divisions.append((i+j))
        
irregular =['CHALLENGERI', 'MASTERI', 'GRANDMASTERI']
for i in irregular:
    all_divisions.append(i)
print(all_divisions)

['DIAMONDI', 'DIAMONDII', 'DIAMONDIII', 'DIAMONDIV', 'PLATINUMI', 'PLATINUMII', 'PLATINUMIII', 'PLATINUMIV', 'GOLDI', 'GOLDII', 'GOLDIII', 'GOLDIV', 'SILVERI', 'SILVERII', 'SILVERIII', 'SILVERIV', 'IRONI', 'IRONII', 'IRONIII', 'IRONIV', 'CHALLENGERI', 'MASTERI', 'GRANDMASTERI']


In [ ]:
#Fetch game IDs for each player's last game, inserting into a new column 'gameId'
watcher = LolWatcher(api_key)
df = pd.DataFrame(ranked_games_by_division_by_tier[tier])
for i, row in df_rg.iterrows():
    try:
        accountId = watcher.summoner.by_name(region, df['summonerName'].values[i])['accountId']
        games = watcher.match.matchlist_by_account(region, accountId)['matches']
        gameId = '%.0f' %[games[x]['gameId'] for x in range(0, len(games)) if games[x]['queue']==420][0]
        df.loc[i, 'gameId'] = gameId
    except: 
        df.loc[i, 'gameId'] = ''
df.gameId.replace('', np.nan, inplace=True)
df.to_csv(tier.replace(" ", "")+'.csv', index=True)
    

In [ ]:
print("For",df_rg['tier'].values[0],df_rg['rank'].values[0],"there are",df_rg['gameId'].nunique(),"unique games")
print("We will now fetch match-specific information for each of those",df_rg['gameId'].nunique(),"games")

In [8]:
from models.liam_gg import game_info_by_match_id

In [9]:
#Doing this for every division will take ~1 week of running constantly
for tier in all_divisions:
    this_df = pd.DataFrame()
    file_name = 'data/league/'+tier.replace(" ", "")+".csv"
    df_GameID = pd.read_csv(file_name)
    df_GameID['gameId'].replace('', np.nan, inplace=True)
    print(file_name)
    for i, row in df_GameID.iterrows():
        if df_GameID['gameId'].values[i] != np.nan:
            try:
                GameID = int(df_GameID['gameId'].values[i])
                Name = df_GameID['summonerName'].values[i]
                df = game_info_by_match_id(api_key, Name, region, 'CLASSIC', GameID).match_data()
                df = df.loc[df['summonerName'] == Name]
                this_df = this_df.append(df)
                time.sleep(1)
            except: 
                continue;
    this_df.to_csv(tier.replace(" ", "")+'_MatchData'+'.csv', index=True)

data/league/SILVERII.csv
data/league/SILVERIII.csv
data/league/SILVERIV.csv
data/league/IRONI.csv
data/league/IRONII.csv
data/league/IRONIII.csv
data/league/IRONIV.csv
